# Isogrid Module Design for the LV4 Sub-Orbital Rocket
## Motivation
PSAS has used carbon fiber composites for generations of rockets. The introduction of isogrid aluminum pressure vessels has granted the impetus to examine isogrid's application as a general structural component. The goal is to have a design that is manufacturable by the amatuer group, but has a mass close to composite designs.

## Design Choices
### Material
For simplicity, the tank is built from a tube or pipe, eliminating the need to weld the length of the tank. The downside is that this limits the material options to what can be obtained off the shelf. A quick survey of choices found the following options, all made from 6061-T6. Note, material prices as of 2020/10

| ID (in) | Wall (in) | OD (in) | Type | Price/Ft | Source |
|---------|-----------|---------|------|----------|--------|
| 10.02 | 0.365 | 10.75 | Pipe | 270.01 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/10-nom-schedule-40-aluminum-pipe-6061-t6-extruded/pid/10619)|
| 10.02 | 0.365 | 10.75 | Pipe | 64.33 | [Midwest Steel & Al*](https://www.midweststeelsupply.com/store/6061aluminumpipe)|
| 10.50 | 0.750 | 12.00 | Tube | 826.71 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/12-od-x-0-75-wall-x-10-5-id-aluminum-round-tube-6061-t6-extruded-aluminum-association-in-/pid/21986)|
| 11.00 | 0.500 | 12.00 | Tube | 678.12 | [Metals Depot](https://www.metalsdepot.com/search?q=T3R12500)
| 11.50 | 0.250 | 12.00 | Tube | 181.63 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/12-od-x-0-25-wall-x-11-5-id-aluminum-round-tube-6061-t6-extruded/pid/23929)|
| 12.00 | 0.375 | 12.75 | Pipe | 85.10 | [Midwest Steel & Al*](https://www.midweststeelsupply.com/store/6061aluminumpipe)|

*For Midwest Steel and Al, material is considered raw and can have light scratches

Other sizes appear to be available from [Vita Needle](https://www.vitaneedle.com/aluminum-tube/) but quotes have to be requested.

6061-T6 is a an acceptable choice as it has reasonable strength, weldability and LOX compatibility. The 11.5 in inner diameter (ID) pipe was selected as it's closest to the initial LV4 multi-disciplinary design targets and least expensive option.

For initial estimates, a total tank length of 48" is assumed. This fits with available manufacuring possibilities.

In [1]:
# Stock properties
grid_id = 12     # in
grid_wall_t = 0.25  # in
grid_od = grid_id+2*grid_wall_t # in
grid_r = grid_od/2   # in
grid_length = 34  # in
offset = 0.2

# Material properties for Al 6061-T6
E_Si = 69.0 # Young's Modules, GPa
density = 0.0975 #lb/in^3
E = 10.1e6 # psi
S_y = 34e3 # Yield compressive Strength - psi
S_u = 45e3 # Ultimate Tensile Strength - psi

nu = 0.33 # Poisson's Ratio


### Safety Factor
Minimum safety factor for Base 11 is 1.5 (see requirement 3.2.4.15)

In [2]:
safety_factor = 1.5


### Pattern size
For optimal strength and to maintain isotropic properties, the isogrid cells need to wrap around the tank diameter in integer increments. Twenty four cells was chosen as a starting point.

In [3]:
import math
num_radial_divisions = 12
pattern_size = grid_od*math.pi/num_radial_divisions
pattern_height = pattern_size*(3**(1/2))/2
print('Pattern Size: {:5.5f}'.format(pattern_size))
print('Pattern Height: {:5.5f}'.format(pattern_height))


Pattern Size: 3.27249
Pattern Height: 2.83406


### Design Properties

from "Isogrid Definitions", page viii

**Design choices:**<br>
t: skin thickness; &emsp;&emsp; minimum 0.02" per Peter<br> 
b: rib width; &emsp;&emsp;&emsp;&emsp;  min 0.03" per Peter<br>
d: rib depth; &emsp;&emsp;&emsp;&emsp; calculated from OD minus skin<br>
c: flange depth<br>
w: flange width<br>
s: plate thickness of unflanged isogrid<br>
h: triangle height<br>
a: triangle leg length (center to center along pattern)<br><br>

**Non-dimensional parameters:**<br>
δ: rib width to skin thickness ratio <br>
λ: flange depth to skin thickness ratio<br>
α: rib cross section to skin cross section ratio<br>
μ: flange cross section to skin cross section ratio<br>
β^2 = bending stiffness parameter<br><br>

**Equivalent material properties:**<br>
t_eff = equivalent thickness for membrane stresses<br>
t_bar = equivalent weight thickness <br>
t_star = equivalent monocoque thickness<br>
E_star = equivalent monocoque Young's modulus<br><br>

**Stiffness coefficients:**<br>
K: extensional stiffness<br>
D: bending stiffness<br>
I: moment of inertia


In [4]:
#Design choices
t = 0.025
b = 0.025
#d = grid_wall_t-t-offset
d = 0.175
c = 0.0
w = 0.0
s = t + d
h = pattern_height
a = pattern_size

# non dimensional terms α λ μ δ β
δ = d/t
λ = c/t
α = b*d/(t*h)
μ = w*c/t/h
β_squared = (1+α+μ)*(3*(1+δ)**2+3*μ*(1+λ)**2+1+α*δ**2+μ*λ**2) - 3*(1+δ-μ*(1+λ))**2
β = β_squared**(1/2)

#Equivalent properties
t_eff = t*(1+α+μ)
t_bar = t*(1+3*(α+μ))
t_star = t*β/(1+α+μ)
E_star = E*(1+α+μ)**2/β

## Tank Properties
The isogrid properties are calculated from NASA CR-124075

In [5]:
print('Rib depth: {:5.3f}'.format(d))
print('Web non-dimensional ratio, Alpha: {:5.3f}'.format(α))
print('Rib depth to skin thickness, Delta: {:5.3f}'.format(δ))
print('Bending stiffness parameter, Beta: {:5.3f}'.format(β))
print('Skin thickness, t: {:5.3f} in'.format(t))
print('Effective thickness, t_eff: {:5.3f} in'.format(t_eff))
print('Equivalant monocoque thickness, t*: {:5.3f} in'.format(t_star))
print('Equivalant weight thickness, t_bar: {:5.3f} in'.format(t_bar))
print('Equivalant monocoque Young\'s Modulus, E*: {:5.3f} psi'.format(E_star))
#print('Equivalant moment of Inertia, I: {:5.3f} in^4'.format(I))
#print('Equivalant beta sq amoment of Inertia, I: {:5.10f} in^4'.format(I_beta))

Rib depth: 0.175
Web non-dimensional ratio, Alpha: 0.062
Rib depth to skin thickness, Delta: 7.000
Bending stiffness parameter, Beta: 4.016
Skin thickness, t: 0.025 in
Effective thickness, t_eff: 0.027 in
Equivalant monocoque thickness, t*: 0.095 in
Equivalant weight thickness, t_bar: 0.030 in
Equivalant monocoque Young's Modulus, E*: 2834960.978 psi


## Mass Model
Equations for calculating the mass of the isotank based on diameter and length

tbar: equiv wt thickness<br>

In [6]:
import math

cross_section = math.pi*((grid_id/2+t_bar)**2-(grid_id/2)**2)
volume = cross_section*grid_length
grid_mass = volume*density
lbm_to_kg = 0.453592 #unit conversion
grid_mass_kg = grid_mass * lbm_to_kg

print('grid_mass:',grid_mass, 'lbm')
print('grid_mass_kg:',grid_mass_kg, 'kg')
#print('tank_wall_mass:',tank_wall_mass, 'kg')

grid_mass: 3.7122260212230436 lbm
grid_mass_kg: 1.6838360254186027 kg


## Load Calculations



### Combined Compression and Bending
For the fuel tank, the cylinder is under compression and bending. Section 4.2 on page 4.2.001 (#97) covers this scenario. The example covers:
* General Instability (N_cr_1)
* Skin Buckling (N_cr_2)
* Rib Crippling (N_cr_3)

In [7]:
import math

# Inputs
F = -10000 # 1573 lbf ~7000 N
M = 0 # 265 lbf*in = 30 Nm

# Instability
Y = 1-0.407*(1-math.exp((-1/16)*(grid_r/t_eff)**(1/2))) #buckling knockdown factor: https://www.hindawi.com/journals/ijae/2020/9851984/#B4
Y_con = 0.2
c0 = 0.612*Y  #4.2.005
c0_con = 0.612*Y_con #extremely conservative
c1 = 10.2     # 4.2.006
c2 = 0.616    # 4.2.008

R_t_star = grid_r/t_star
L_R = grid_length/grid_r

# Classical Value
N_cl = (1/(3*(1-nu**2))**0.5)*E_star*t_star**2/grid_r # k/in

N_a = F/(2*math.pi*grid_r) # k/in
N_b = M/(math.pi*grid_r**2) # k/in
N_cr = N_a + N_b
Ncr_Nb_check = N_cr/4
N_cr2 = E_star*t_star**2*(3*(1-nu)**2)**(-1/2)/grid_r

euler = 0.5 #(0.5 is a conservative estimate given it is unlikely our L/R ratios exceed 10. see 4.2.003)
N_cr_timo = N_cl*euler
N_cr_KDL = N_cl*Y

N_cr_1 = c0*E*t**2*β/grid_r
N_cr_1_con = c0_con*E*t**2*β/grid_r
#N_cr_2 = c1*E_star*t_star*(1+α)*(t_star**2)/(h**2)
k_c = 9 + 2/(h/a-0.2)**1.6 #shear buckling fixed edge: https://www.abbottaerospace.com/aa-sb-001/15-local-stability-isotropic-materials/15-2-general-buckling-expression/15-2-8-buckling-of-triangular-panels/
K_c = k_c*(math.pi**2)/(12*(1-nu**2))
N_cr_2b = K_c*E*(t/h)**2
N_cr_2 = c1*E*t*(1+α)*(t/h)**2
N_cr_3 = c2*E*t*(1+α)*(b/d)**2


print('Axial load per inch (Na): {:5.1f} lbf/in'.format(N_a))
print('Max moment per inch (Nb): {:5.1f} lbf/in'.format(N_b))
print('Ncr - ',N_cr,'lbf/in')
print('Ncr_Nb_check ',Ncr_Nb_check,'\n')


print('If Ncr_Nb_check > Nb: \n edit Euler modifier ({:1.2f}) using ref 2.1 (4.2.002) \n https://sci-hub.se/https://doi.org/10.1017/S0001924000014755'.format(euler))
print('Euler buckling lookup values:\n R/t*: {:5.1f} \n L/R: {:5.1f}'.format(R_t_star,L_R))

print('\nBuckling: \n classical (Nb > check value above): \n  {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cl,N_cl*2*math.pi*grid_r))
print(' conservative (Euler): \n  {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cr_timo,N_cr_timo*2*math.pi*grid_r))
print(' calculated (KDF = {:1.2f}) \n  ref: https://www.hindawi.com/journals/ijae/2020/9851984/#B4\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(Y,N_cr_KDL,N_cr_KDL*2*math.pi*grid_r))
print('\nGeneral Instability: \n {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cr_1,N_cr_1*2*math.pi*grid_r))
print('General Instability (conservative KDF = {:1.2f} from 4.2.005): \n {:5.1f} lbf/in || {:5.1f} lbf'.format(Y_con,N_cr_1_con,N_cr_1_con*2*math.pi*grid_r))
### I realized that General Instability and Buckling are the same thing; not sure which of these apply, probably best to look at the most conservative or analyze in FEA.

print('Skin Buckling: \n {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cr_2,N_cr_2*2*math.pi*grid_r))
print('K_c: \n {:5.1f} lbf'.format(K_c))
print('Skin Buckling2: \n {:5.1f} lbf'.format(N_cr_2b))
print('Rib Crippling: \n {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cr_3,N_cr_3*2*math.pi*grid_r))



Axial load per inch (Na): -254.6 lbf/in
Max moment per inch (Nb):   0.0 lbf/in
Ncr -  -254.64790894703253 lbf/in
Ncr_Nb_check  -63.66197723675813 

If Ncr_Nb_check > Nb: 
 edit Euler modifier (0.50) using ref 2.1 (4.2.002) 
 https://sci-hub.se/https://doi.org/10.1017/S0001924000014755
Euler buckling lookup values:
 R/t*:  66.1 
 L/R:   5.4

Buckling: 
 classical (Nb > check value above): 
  2480.9 lbf/in || 97426.1 lbf
 conservative (Euler): 
  1240.5 lbf/in || 48713.0 lbf
 calculated (KDF = 0.75) 
  ref: https://www.hindawi.com/journals/ijae/2020/9851984/#B4
  1858.2 lbf/in || 72970.8 lbf

General Instability: 
 1859.4 lbf/in || 73017.1 lbf
General Instability (conservative KDF = 0.20 from 4.2.005): 
 496.5 lbf/in || 19497.6 lbf
Skin Buckling: 
 212.8 lbf/in || 8356.1 lbf
K_c: 
  11.8 lbf
Skin Buckling2: 
 9308.4 lbf
Rib Crippling: 
 3370.3 lbf/in || 132351.1 lbf


## Stress Calcs

In [8]:
Fx = F #axial
Fy = 0 #circumferential
Txy = 0.0

Na = F/(math.pi*grid_id)
Nb = M/(math.pi*(grid_id/2)**2)
N_max = Na+N_b
N_min = Na-Nb

σ_skin_max  = N_max/t_eff #(2.0.021)

σ_1_rib = (-1/(3*t_eff))*(Na+Nb)
σ_2_3_rib = (2/(3*t_eff))*(Na+Nb)


#add in membrane rib and skin stresses (2.0.021)

print('Loads:\n axial:\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(Na,Na*2*math.pi*grid_r))
print(' bending:\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(Nb,Nb*2*math.pi*grid_r))
print(' combined max:\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(N_max,N_max*math.pi*grid_id))
print(' combined min:\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(N_min,N_min*math.pi*grid_id))
print('\nMembrane skin stress:')
print(' σ max: {:5.1f} psi'.format(σ_skin_max))
print('\nMembrane rib stress:')
print(' σ_1 rib: {:5.1f} psi'.format(σ_1_rib))
print(' σ_2_3 rib: {:5.1f} psi'.format(σ_2_3_rib))

Loads:
 axial:
  -265.3 lbf/in || -10416.7 lbf
 bending:
    0.0 lbf/in ||   0.0 lbf
 combined max:
  -265.3 lbf/in || -10000.0 lbf
 combined min:
  -265.3 lbf/in || -10000.0 lbf

Membrane skin stress:
 σ max: -9993.3 psi

Membrane rib stress:
 σ_1 rib: 3331.1 psi
 σ_2_3 rib: -6662.2 psi
